In [3]:
# import torch
# from torchvision import datasets,transforms
# from torch.utils.data import DataLoader, SubsetRandomSampler
# import torch.nn as nn
# import torch.nn.functional as F
# import numpy as np
# import matplotlib.pyplot as plt

In [4]:
# transform_train = transforms.Compose([
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(10),
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
# ])

# # Normalize the test set same as training set without augmentation
# transform_test = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
# ])

In [ ]:
# train_data=datasets.CIFAR10('data',train=True,download=True,transform=transform_train)
# test_data=datasets.CIFAR10('data',train=False,download=True,transform=transform_test)

In [6]:

# num_workers=0
# batch_size=50
# valid_size=0.2
# train_length = len(train_data)
# indices=list(range(len(train_data)))
# split = int(np.floor(valid_size * train_length))

# np.random.shuffle(indices)

# train_idx=indices[split:]
# valid_idx=indices[:split]

# train_sampler=SubsetRandomSampler(train_idx)
# validation_sampler=SubsetRandomSampler(valid_idx)

# train_loader=DataLoader(train_data,num_workers=num_workers,batch_size=batch_size,sampler=train_sampler)
# valid_loader=DataLoader(train_data,num_workers=num_workers,batch_size=batch_size,sampler=validation_sampler)
# test_loader=DataLoader(test_data,shuffle=True,num_workers=num_workers,batch_size=batch_size)

In [ ]:
# classes=['airplane', 'automobile', 'bird', 'cat', 'deer',
#            'dog', 'frog', 'horse', 'ship', 'truck']

# def RGBshow(img):
#     img=img*0.5+0.5
#     plt.imshow(np.transpose(img,(1,2,0)))

# fig=plt.figure(1,figsize=(30,5))
# for idx in range(batch_size):
#     ax=fig.add_subplot(2,batch_size/2,idx+1,xticks=[],yticks=[])
#     RGBshow(images[idx])
#     ax.set_title(classes[labels[idx]])

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define the mean and standard deviation for CIFAR-10
# These values are precomputed for the CIFAR-10 dataset
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)  # Mean of the CIFAR-10 dataset
CIFAR10_STD = (0.2470, 0.2435, 0.2616)   # Standard deviation of the CIFAR-10 dataset

# Define transformations for training and testing data
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.RandomCrop(32, padding=4),  # Randomly crop images with padding
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)  # Normalize using CIFAR-10 mean and std
])

test_transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)  # Normalize using CIFAR-10 mean and std
])

# Load the CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(
    root='./data',  # Path to store the dataset
    train=True,  # Load the training set
    download=True,  # Download the dataset if not already present
    transform=train_transform  # Apply training transformations
)

test_dataset = torchvision.datasets.CIFAR10(
    root='./data',  # Path to store the dataset
    train=False,  # Load the test set
    download=True,  # Download the dataset if not already present
    transform=test_transform  # Apply testing transformations
)

# Create data loaders
batch_size = 64  # You can adjust the batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Check the dataset
print(f"Training dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

# Example: Visualize a batch of images
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img = img / 2 + 0.5  # Unnormalize the image
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))  # Convert from (C, H, W) to (H, W, C)
    plt.show()

# Get a batch of training images
dataiter = iter(train_loader)
images, labels = next(dataiter)

# Show images
imshow(torchvision.utils.make_grid(images))
print(' '.join(f'{train_dataset.classes[labels[j]]}' for j in range(batch_size)))

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the CNN model
class CIFAR10CNN(nn.Module):
    def __init__(self):
        super(CIFAR10CNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)  # Input: 32x32x3, Output: 32x32x32
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: 16x16x32
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  # Output: 16x16x64
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: 8x8x64
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)  # Output: 8x8x64

        # Fully connected layers
        self.fc1 = nn.Linear(8 * 8 * 64, 64)  # Input: 8*8*64, Output: 64
        self.fc2 = nn.Linear(64, 10)  # Output: 10 (for 10 classes)

    def forward(self, x):
        # Convolutional layers with ReLU and pooling
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))

        # Flatten the output for the fully connected layers
        x = x.view(-1, 8 * 8 * 64)  # Reshape to (batch_size, 8*8*64)

        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the model
model = CIFAR10CNN()

# Print the model summary
print(model)

CIFAR10CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)


In [ ]:
import torch.optim as optim

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Loss function for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Optimizer

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0

print('Training finished.')

Epoch [1/3], Step [100/782], Loss: 1.5826
Epoch [1/3], Step [200/782], Loss: 1.4805
Epoch [1/3], Step [300/782], Loss: 1.4357
Epoch [1/3], Step [400/782], Loss: 1.3889
Epoch [1/3], Step [500/782], Loss: 1.3996


In [ ]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():  # Disable gradient computation
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)  # Get the predicted class
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test set: {100 * correct / total:.2f}%')